

```
The Notebook can be loaded from this link:
https://colab.research.google.com/drive/11TtbhN4nuvCMn-VqsL65Cf9AZL8yx-yv
```



In [1]:
import os
from getpass import getpass
user = getpass('GitHub user')
password = getpass('GitHub password')
os.environ['GITHUB_AUTH'] = user + ':' + password
!git clone https://$GITHUB_AUTH@github.com/chengmar09/comp0053.git  

GitHub user··········
GitHub password··········
Cloning into 'comp0053'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 46 (delta 20), reused 30 (delta 10), pack-reused 0
Unpacking objects: 100% (46/46), done.


In [2]:
!echo 'Check Current Dir'
%cd /content/comp0053/
!pwd
!ls
!pip install -r Requirement.txt
!pip install h5py
%tensorflow_version 1.x

Check Current Dir
/content/comp0053
/content/comp0053
COMP0053_EmoPain_Demo.ipynb  helper.py	Requirement.txt
data			     README.md	train.py
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=1dbe101e013082e1db16e0a381dc1e8438ae0696337588a1c1d648cabe32dbf5
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
TensorFlow 1.x selected.


In [3]:
!pwd
!ls

/content/comp0053
COMP0053_EmoPain_Demo.ipynb  helper.py	Requirement.txt
data			     README.md	train.py


In [4]:
from helper import *
import scipy

movementDataLoader = DataReader('Movementdata') #<- Automatically download data
#train_data, valid_data = movementDataLoader.get_data()
#movementDataLoader.get_weights(data='Movementdata', name='stack_bilstm.h5')

(train_x, train_y), (valid_x, valid_y) = movementDataLoader.more_more_processing(v_stack=False)

def _data_reshaping(X_tr, X_va, network_type):
    _, win_len, dim = X_tr.shape
    print(network_type)
    if network_type=='CNN' or network_type=='ConvLSTM':

        # make it into (frame_number, dimension, window_size, channel=1) for convNet
        X_tr = np.swapaxes(X_tr,1,2)
        X_va = np.swapaxes(X_va,1,2)

        X_tr = np.reshape(X_tr, (-1, dim, win_len, 1))
        X_va = np.reshape(X_va, (-1, dim, win_len, 1))

    if network_type=='MLP':
        print('MLP...')
        X_tr = np.reshape(X_tr, (-1, dim*win_len))
        X_va = np.reshape(X_va, (-1, dim*win_len))

    return X_tr, X_va


# (n, 2, 30)
def _shape_changing(x_tr, y_tr, x_va, y_va): #LSTM

    segment_size = 30
    
    NUM_SAMPLES_TR = int(len(x_tr) /segment_size)
    NUM_SAMPLES_VA = int(len(x_va) /segment_size)

    new_train_x = []
    new_train_y = []
    for i in range(NUM_SAMPLES_TR-segment_size-1):

        temp_x = x_tr[i:i+segment_size]
        temp_y = y_tr[i+segment_size]

        new_train_x.append(temp_x)
        new_train_y.append(temp_y)

    new_valid_x = []
    new_valid_y = []
    for i in range(NUM_SAMPLES_VA-segment_size-1):

        temp_x = x_va[i:i+segment_size]
        temp_y = y_va[i+segment_size]

        new_valid_x.append(temp_x)
        new_valid_y.append(temp_y)

    new_train_x = np.array(new_train_x)
    new_train_y = np.array(new_train_y)

    new_valid_x = np.array(new_valid_x)
    new_valid_y = np.array(new_valid_y)

    return (new_train_x, new_train_y), (new_valid_x, new_valid_y)

(train_x, train_y), (valid_x, valid_y) = _shape_changing(train_x, train_y, valid_x, valid_y)

print(train_x.shape, train_y.shape)
print(valid_x.shape, valid_y.shape)

Using TensorFlow backend.


Create a ->Movementdata<- folder 

Now downloading training data

Now downloading Validation data
Unzipping training data
Unzipping validation data
There is total of 23 training data
There is total of 12 Validation data
(11839, 30, 26) (11839, 1)
(5250, 30, 26) (5250, 1)


In [5]:
import keras
num_classes = 2
train_y = keras.utils.to_categorical(train_y, num_classes)
valid_y = keras.utils.to_categorical(valid_y, num_classes)

from sklearn.metrics import f1_score

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Input, GRU
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Bidirectional(LSTM(units=32, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=32, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=32)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', f1_m])

# checkpoint
checkpoint_path = "stack_bilstm.h5"

#model = load_model(checkpoint_path, custom_objects={'f1_score': f1_score})
checkpoint = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False, verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

history = model.fit(train_x, train_y, validation_split=0.1, epochs=5, callbacks=callbacks_list)  # starts training






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 10655 samples, validate on 1184 samples
Epoch 1/5





10655/10655 [==============================] - 45s 4ms/step - loss: 0.1816 - acc: 0.9435 - f1_m: 0.9422 - val_loss: 0.5543 - val_acc: 0.7576 - val_f1_m: 0.7576

Epoch 00001: val_loss improved from inf to 0.55427, saving model to stack_bilstm.h5
Epoch 2/5
10655/10655 [==============================] - 39s 4ms/step - loss: 0.0842 - acc: 0.9668 - f1_m: 0.9668 - val_loss: 0.4698 - val_acc: 0.8936 - val_f1_m: 0.8757

Epoch 00002: val_loss improved from 0.55427 to 0.46977, saving model to stack_bilstm.h5
Epoch 3/5
10655/10655 [==============================] - 39s 4ms/step - loss: 0.0708 - acc: 0.9711 - f1_m: 0.9704 - val_loss: 0.5121 - val_acc: 0.8809 - val_f1_m: 0.7377

Epoch 00003: val_loss did not improve from 

In [6]:
from keras.models import load_model

model = load_model(checkpoint_path, custom_objects={'f1_m': f1_m})
# summarize model.
model.summary()

#model.load_weights('best_model.hdf5')
validation_score = model.evaluate(valid_x, valid_y)

for name, score in zip(model.metrics_names, validation_score):
    print(f'{name} : {score}')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 30, 64)            15104     
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 64)            0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 30, 64)            24832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 64)            0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 64)                24832     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                

/content/comp0053
COMP0053_EmoPain_Demo.ipynb  helper.py	  README.md	   stack_bilstm.h5
data			     __pycache__  Requirement.txt  train.py
[master b62736b] Update helper from colab
 1 file changed, 17 insertions(+), 11 deletions(-)
Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 437 bytes | 437.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/chengmar09/comp0053.git
   f1b78c2..b62736b  master -> master
